# Simulation Boxes

## Overview

The `Box` class represents the simulation cell that defines spatial boundaries and periodic boundary conditions (PBC) for molecular systems. It serves as the foundation for coordinate transformations, distance calculations, and integration with molecular dynamics engines.

**Design Philosophy**

Box uses a 3×3 matrix representation where columns are lattice vectors—a convention compatible with LAMMPS, GROMACS, and other MD engines. This design allows Box to handle three geometric styles automatically:

- **FREE**: Zero-volume box (no boundaries)
- **ORTHOGONAL**: Rectangular box with diagonal matrix
- **TRICLINIC**: General parallelepiped with off-diagonal elements

The class inherits from `PeriodicBoundary`, integrating with MolPy's region system for spatial queries and filtering. Box automatically detects its style from the matrix structure, enabling polymorphic behavior in coordinate operations.

**Integration with MolPy**

Box is used by `Frame` to define system boundaries, by trajectory readers to parse box information from MD files, and by analysis tools for PBC-aware distance calculations. The matrix-based representation ensures seamless conversion to/from engine-specific formats.

---


## Creating Boxes

Box provides factory methods for common geometries. These methods accept optional `pbc` (periodic boundary conditions per axis), `origin` (box origin offset), and `central` (center box at origin) parameters.

The underlying representation is always a 3×3 matrix, but factory methods simplify common cases and ensure correct matrix structure.


In [1]:
import molpy as mp
import numpy as np

# Cubic box: all sides equal (most common for simple systems)
cubic = mp.Box.cubic(20.0)  # 20 Å cube
print(f"Cubic: {cubic}")  # Auto-detected as ORTHOGONAL style

# Orthogonal box: rectangular with different side lengths
ortho = mp.Box.orth([10.0, 20.0, 30.0])
print(f"Orthogonal: {ortho}")

# Triclinic box: general parallelepiped (for non-orthogonal crystals)
# Can be created from lengths + tilts (LAMMPS convention)
tric = mp.Box.tric(
    lengths=[10.0, 12.0, 15.0],
    tilts=[1.0, 0.5, 0.2]  # xy, xz, yz tilt factors
)
print(f"Triclinic: {tric}")

# Or from lengths + angles (crystallographic convention)
tric2 = mp.Box.from_lengths_angles(
    lengths=[10.0, 12.0, 15.0],
    angles=[90.0, 90.0, 120.0]  # alpha, beta, gamma in degrees
)

# Direct matrix construction (for full control)
matrix = np.array([[10.0, 1.0, 0.5], [0.0, 12.0, 0.2], [0.0, 0.0, 15.0]])
box_from_matrix = mp.Box(matrix=matrix)

Cubic: <Orthogonal Box: [20. 20. 20.]>
Orthogonal: <Orthogonal Box: [10. 20. 30.]>
Triclinic: <Triclinic Box: [10.         12.04159458 15.00966355], [1.  0.5 0.2]>


## Periodic Boundary Conditions

PBC control whether coordinates wrap around box boundaries. This is essential for bulk simulations but can be disabled per axis for surfaces, interfaces, or confined systems. Box tracks PBC independently for x, y, z axes, allowing mixed boundary conditions (e.g., periodic in xy, fixed in z for slab geometries).


In [2]:
# Default: periodic in all dimensions
box = mp.Box.cubic(10.0)
print(f"Default PBC: {box.pbc}")  # [True, True, True]

# Slab geometry: periodic in X and Y, fixed in Z
slab = mp.Box.orth([20.0, 20.0, 50.0], pbc=[True, True, False])
print(f"Slab PBC: {slab.pbc}")  # [True, True, False]

# Modify PBC after creation
slab.periodic_z = True  # Enable PBC in Z
slab.periodic = [False, False, False]  # Disable all PBC
print(f"Modified: {slab.pbc}")

Default PBC: [ True  True  True]
Slab PBC: [ True  True False]
Modified: [False False False]


## Box Properties and Geometry

Box exposes geometric properties computed from the matrix. The `style` property automatically detects the box type, enabling style-specific optimizations in coordinate operations. Key properties include dimensions (`lengths`, `volume`), matrix representation (`matrix`), and boundary information (`bounds`, `origin`).


In [3]:
box = mp.Box.orth([10.0, 12.0, 15.0])

# Style detection (affects coordinate operation algorithms)
print(f"Style: {box.style}")  # Style.ORTHOGONAL

# Geometric properties
print(f"Lengths: {box.lengths}")  # [10.0, 12.0, 15.0]
print(f"Volume:  {box.volume:.2f} Å³")  # 1800.0
print(f"Origin:  {box.origin}")  # [0.0, 0.0, 0.0]

# Matrix representation (columns are lattice vectors)
print(f"Matrix:\n{box.matrix}")

# For triclinic boxes, access tilt factors
tric = mp.Box.tric([10.0, 12.0, 15.0], [1.0, 0.5, 0.2])
print(f"Tilts: {tric.tilts}")  # [1.0, 0.5, 0.2] (xy, xz, yz)
print(f"Angles: {tric.angles}")  # Converted to degrees

# Boundary coordinates (useful for visualization/export)
print(f"Bounds:\n{box.bounds}")  # [[xlo, ylo, zlo], [xhi, yhi, zhi]]

Style: Style.ORTHOGONAL
Lengths: [10. 12. 15.]
Volume:  1800.00 Å³
Origin:  [0. 0. 0.]
Matrix:
[[10.  0.  0.]
 [ 0. 12.  0.]
 [ 0.  0. 15.]]
Tilts: [1.  0.5 0.2]
Angles: [89.08064274 88.09101712 85.23635831]
Bounds:
[[ 0.  0.  0.]
 [10. 12. 15.]]


## Coordinate Operations

Box provides coordinate transformations and PBC-aware operations. The fractional coordinate system (0 to 1 in each dimension) simplifies wrapping and image calculations. Box automatically selects the appropriate algorithm based on style: orthogonal boxes use fast direct calculations, while triclinic boxes use matrix transformations.


In [4]:
box = mp.Box.cubic(10.0)

# Wrap coordinates back into box (respects PBC settings)
points = np.array([[12.0, -2.0, 5.0], [25.0, 8.0, -3.0]])
wrapped = box.wrap(points)
print(f"Wrapped:\n{wrapped}")  # All coordinates in [0, 10] range

# Convert between absolute and fractional coordinates
absolute = np.array([[5.0, 3.0, 7.0]])
fractional = box.make_fractional(absolute)
print(f"Fractional: {fractional}")  # [0.5, 0.3, 0.7]

# Convert back
absolute_restored = box.make_absolute(fractional)
print(f"Restored: {absolute_restored}")

# Get image flags (which periodic image a point belongs to)
images = box.get_images(points)
print(f"Image flags:\n{images}")  # [[1, -1, 0], [2, 0, -1]]

# Unwrap coordinates using image flags
unwrapped = box.unwrap(wrapped, images)
print(f"Unwrapped:\n{unwrapped}")  # Original positions restored

Wrapped:
[[2. 8. 5.]
 [5. 8. 7.]]
Fractional: [[0.5 0.3 0.7]]
Restored: [[5. 3. 7.]]
Image flags:
[[ 1 -1  0]
 [ 2  0 -1]]
Unwrapped:
[[12. -2.  5.]
 [25.  8. -3.]]


## Distance Calculations

Box provides PBC-aware distance calculations that automatically handle periodic boundaries. The `diff()` method computes the minimum-image distance vector between two points, while `dist()` returns the scalar distance. These methods work correctly for both orthogonal and triclinic boxes.


In [5]:
box = mp.Box.cubic(10.0)

# Two points that would be far apart without PBC
r1 = np.array([[1.0, 1.0, 1.0]])
r2 = np.array([[9.5, 9.5, 9.5]])

# Minimum-image distance vector (accounts for PBC)
dr = box.diff(r1, r2)
print(f"Distance vector: {dr}")  # [-1.5, -1.5, -1.5] (not [8.5, 8.5, 8.5])

# Scalar distance
distance = box.dist(r1, r2)
print(f"Distance: {distance}")  # 2.598 (not 14.722)

# Pairwise distances between two sets of points
points1 = np.array([[1.0, 1.0, 1.0], [2.0, 2.0, 2.0]])
points2 = np.array([[9.5, 9.5, 9.5], [8.0, 8.0, 8.0]])
distances = box.dist_all(points1, points2)
print(f"Pairwise distances:\n{distances}")  # Shape: (2, 2)


Distance vector: [[1.5 1.5 1.5]]
Distance: [2.59807621]
Pairwise distances:
[[2.59807621 5.19615242]
 [4.33012702 6.92820323]]


## Example: Using Box in a Simulation Workflow

This example demonstrates a realistic workflow: creating a box for a bulk water simulation, checking point containment, and performing PBC-aware analysis.


In [6]:
# Create a cubic box for bulk water simulation
# Target density: ~1 g/cm³, ~1000 water molecules
# Approximate volume: 30×30×30 Å³
box = mp.Box.cubic(30.0, pbc=[True, True, True])
print(f"Box volume: {box.volume:.1f} Å³")

# Generate some atomic positions (example)
n_atoms = 100
positions = np.random.rand(n_atoms, 3) * 30.0  # Random positions in box

# Check which atoms are inside the box
inside = box.isin(positions)
print(f"Atoms inside box: {inside.sum()}/{n_atoms}")

# Wrap any atoms that moved outside (e.g., after MD step)
positions_wrapped = box.wrap(positions)

# Calculate distances between all pairs (PBC-aware)
# This is useful for neighbor lists, RDF calculations, etc.
distances = box.dist_all(positions_wrapped, positions_wrapped)
# Remove self-distances (diagonal)
np.fill_diagonal(distances, np.inf)
min_dist = distances.min()
print(f"Minimum interatomic distance: {min_dist:.3f} Å")

# Modify box size (e.g., NPT simulation)
box.lx = 31.0
box.ly = 31.0
box.lz = 31.0
print(f"New volume: {box.volume:.1f} Å³")
print(f"New density: {n_atoms * 18.015 / (box.volume * 1e-24) / 6.022e23:.3f} g/cm³")


Box volume: 27000.0 Å³
Atoms inside box: 100/100
Minimum interatomic distance: 0.601 Å
New volume: 29791.0 Å³
New density: 0.100 g/cm³
